In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("Adops & Data Scientist Sample Data - Q1 Analytics.csv")

In [3]:
data.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


## Question 1

Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?


In [4]:
# Filter the data on country id and store it in a separate variable

sub_df = data[data.country_id=="BDV"][["site_id","user_id"]]

In [6]:
# Group the data for country_id=BDV on site_id 

grouped_by_site_id = sub_df.groupby(["site_id"]).nunique()

In [14]:
# Sort (in descending order) the grouped data frame on user_id and take the first row

grouped_by_site_id.sort_values('user_id', ascending=False).head(1)

,site_id,user_id
site_id,,
5NPAU,1,544


## Question 2

Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [15]:
# Subset the data based on the timestamp condition

sub_df = data[(data.ts >= '2019-02-03 00:00:00') & (data.ts <= '2019-02-04 23:59:59') ]

In [20]:
sub_df.groupby(["user_id", "site_id"]).count()

ts  country_id
user_id site_id                
LC00C3  5NPAU     1           1
LC01C3  5NPAU     1           1
LC06C3  GVOFK     1           1
        N0OTG    25          25
LC0C3E  N0OTG     1           1
...              ..         ...
LCDFC3  N0OTG     1           1
LCE2C3  5NPAU     3           3
LCE4C3  N0OTG     1           1
LCFC32  5NPAU     1           1
LCFFC3  N0OTG     1           1

[746 rows x 2 columns]

In [21]:
# Group the subset on user_id and site_id. Then take it count.
# Reset the index to make sure user_id appears as a column instead of index
# Sort the resulting dataframe in descending order on ts - this contains how many time user_id "i" visited
#.... a site with site_id "j"

final_df = sub_df.groupby(["user_id", "site_id"]).count().reset_index().sort_values("ts", ascending=False)

In [23]:
# From the resulting data frame, select rows/users which have atleast 10 visits to those site_id's

final_df[final_df.ts > 10]

,user_id,site_id,ts,country_id
417,LC3A59,N0OTG,26,26
3,LC06C3,N0OTG,25,25
493,LC3C9D,N0OTG,17,17
485,LC3C7E,3POLC,15,15


## Question 3

For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

In [32]:
# Group the data on user_id and get the last timestamp for every user_id
user_id_with_timestamp = data.groupby(["user_id"])["ts"].max().reset_index()

In [33]:
user_id_with_timestamp.head(5)

,user_id,ts
0,LC00C3,2019-02-03 18:52:50
1,LC01C3,2019-02-04 11:35:10
2,LC05C3,2019-02-02 14:14:44
3,LC06C3,2019-02-07 01:16:12
4,LC07C3,2019-02-05 19:06:42


In [36]:
# Merge the resulting dataframe on the actual data on user_id and ts
# This merge helps us to get the country_id and also the site_id 
# We needed to merge with the actual data to get the site_id which eventually has to be returned

merged_with_data_dataframe = user_id_with_timestamp.merge(data, on=["user_id","ts"])

In [40]:
merged_with_data_dataframe.head(5)

,user_id,ts,country_id,site_id
0,LC00C3,2019-02-03 18:52:50,QLT,5NPAU
1,LC01C3,2019-02-04 11:35:10,QLT,5NPAU
2,LC05C3,2019-02-02 14:14:44,BDV,5NPAU
3,LC06C3,2019-02-07 01:16:12,TL6,N0OTG
4,LC07C3,2019-02-05 19:06:42,BDV,5NPAU


In [39]:
# Check if the merged data frame has the same shape with respect to the dataframe user_id_with_timestamp
# ... which was created before merging

user_id_with_timestamp.shape, merged_with_data_dataframe.shape

((1916, 2), (1916, 4))

In [44]:
# Group the merged data frame with site_id and then take a count on either "ts" or "user_id" - lets take "ts"
# sort the count values in descending order and take the top 3

merged_with_data_dataframe.groupby("site_id").ts.count().reset_index().sort_values("ts", ascending=False).head(3)

,site_id,ts
1,5NPAU,992
5,N0OTG,561
6,QGO3G,289


## Question 4

For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [45]:
# Create two data frames by grouping on user_id
# one for ther first site the user visited - min_ts_df
# other for the last site the user visited - max_ts_df

min_ts_df = data.groupby(["user_id"])["ts"].min().reset_index()
max_ts_df = data.groupby(["user_id"])["ts"].max().reset_index()

In [47]:
min_ts_df.head(5)

,user_id,ts
0,LC00C3,2019-02-03 18:52:50
1,LC01C3,2019-02-04 11:35:10
2,LC05C3,2019-02-02 14:14:44
3,LC06C3,2019-02-01 22:49:39
4,LC07C3,2019-02-05 19:06:42


In [50]:
# Merge the min_ts_df, max_ts_df with the actual data frame on user_id and ts
# We need to merge with. the actual data to get the site id

min_ts_data_merged = min_ts_df.merge(data, on = ["user_id", "ts"])
max_ts_data_merged = max_ts_df.merge(data, on=["user_id", "ts"])

In [51]:
min_ts_data_merged.head(5)

,user_id,ts,country_id,site_id
0,LC00C3,2019-02-03 18:52:50,QLT,5NPAU
1,LC01C3,2019-02-04 11:35:10,QLT,5NPAU
2,LC05C3,2019-02-02 14:14:44,BDV,5NPAU
3,LC06C3,2019-02-01 22:49:39,TL6,N0OTG
4,LC07C3,2019-02-05 19:06:42,BDV,5NPAU


In [52]:
max_ts_data_merged.head(5)

,user_id,ts,country_id,site_id
0,LC00C3,2019-02-03 18:52:50,QLT,5NPAU
1,LC01C3,2019-02-04 11:35:10,QLT,5NPAU
2,LC05C3,2019-02-02 14:14:44,BDV,5NPAU
3,LC06C3,2019-02-07 01:16:12,TL6,N0OTG
4,LC07C3,2019-02-05 19:06:42,BDV,5NPAU


In [53]:
# merge the new min_ts_data_merged with max_ts_data_merged with user_id as the key

merged_min_max_with_data = max_ts_data_merged.merge(min_ts_data_merged, on="user_id")

In [54]:
merged_min_max_with_data.head(5)

,user_id,ts_x,country_id_x,site_id_x,ts_y,country_id_y,site_id_y
0,LC00C3,2019-02-03 18:52:50,QLT,5NPAU,2019-02-03 18:52:50,QLT,5NPAU
1,LC01C3,2019-02-04 11:35:10,QLT,5NPAU,2019-02-04 11:35:10,QLT,5NPAU
2,LC05C3,2019-02-02 14:14:44,BDV,5NPAU,2019-02-02 14:14:44,BDV,5NPAU
3,LC06C3,2019-02-07 01:16:12,TL6,N0OTG,2019-02-01 22:49:39,TL6,N0OTG
4,LC07C3,2019-02-05 19:06:42,BDV,5NPAU,2019-02-05 19:06:42,BDV,5NPAU


In [55]:
# Now we have a data frame where we have the site_id for the first visit and the last visit for every user_id.
# Get the number of records where the site_id is the same for the first and the last visit for every user_id. 

merged_min_max_with_data[merged_min_max_with_data.site_id_x == merged_min_max_with_data.site_id_y].shape[0]

1670